In [ ]:
from loguru import logger
from request import GenerateRequest
import asyncio
from cli_client import AspectRatio, ar_map, send_req, GenError
from utils import image_from_bytes
from result import Result, Ok, Err
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

In [ ]:
host = "127.0.0.1:7000"
sub_folder = ""
prompt = "fkey, 1girl black serafuku"
w = None
h = None
ar = AspectRatio.UltraTall
batch_count = 4
if ar is not None:
    if w is not None and h is not None:
        display("Both width and height are specified, ignoring aspect ratio")
    else:
        w, h = ar_map[ar]
        display(f"Using aspect ratio {ar.name} ({w}x{h})")
if w is None or h is None:
    display("Width or height is not specified, using default 1024x1024")
    w = 1024
    h = 1024

req = GenerateRequest(
    prompt=prompt,
    neg_prompt="bad quality",
    seed=-1,
    scale=5.0,
    width=w,
    height=h,
    steps=28,
    sampler="k_euler",
    schedule="native",
    smea=False,
    dyn=False,
    dyn_threshold=False,
    cfg_rescale=0,
)

In [ ]:
total_timeout = batch_count * 30

async def run() -> Result[bytes, GenError]:
    promise = asyncio.gather(*[
        send_req(host, req, sub_folder, total_timeout)
        for _ in range(batch_count)
    ])
    await promise
    return promise.result()

results = await run()

In [ ]:
for result in results:
    match result:
        case Ok(data):
            image = image_from_bytes(data)
            plt.figure()
            plt.imshow(image)
        case Err(err):
            display(err)